In [1]:
# Imports.
import sys
print(sys.version)
print()
from collections import deque
import matplotlib.pyplot as plt
import gym
import random
import time
import numpy as np
from tqdm.notebook import trange
import tensorflow as tf
print("Tensorflow", tf.__version__)
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
print(tf.config.list_physical_devices('GPU'))
print()
import slimevolleygym

3.6.9 (default, Jan 26 2021, 15:33:00) 
[GCC 8.4.0]

Tensorflow 2.5.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]



In [2]:
from tf_agents.agents.dqn import dqn_agent
from tf_agents.environments import suite_gym
from tf_agents.environments import py_environment
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.environments import utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.specs import tensor_spec
from tf_agents.utils import common
from tf_agents.specs import array_spec
from tf_agents.trajectories import time_step as ts

In [3]:
from IPython import display
from pyvirtualdisplay import Display
displayy = Display(visible=0, size=(1, 1))
displayy.start()
%matplotlib inline

# Parameters

In [4]:
num_iterations = 90000 # @param {type:"integer"}

initial_collect_steps = 100  # @param {type:"integer"} 
collect_steps_per_iteration = 1  # @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 200  # @param {type:"integer"}

num_eval_episodes = 20  # @param {type:"integer"}
eval_interval = 1000  # @param {type:"integer"}

# Create Environment

In [5]:
# Create TF Environment around SlimeVolleyball.
class TFSlimeVolley(py_environment.PyEnvironment):
    def __init__(self):
        #self._action_spec = array_spec.BoundedArraySpec(shape=(), dtype=np.int8, name='action', minimum=0, maximum=1)
        #self._observation_spec = ts.TimeStep({'discount': array_spec.BoundedArraySpec(
        #    shape=(), dtype=np.float32, name='discount', minimum=0, maximum=1),
        #                                     'observation': array_spec.BoundedArraySpec(
        #    shape=(12,), dtype=np.float32, name='observation', minimum=np.finfo(np.float32).min, maximum=np.finfo(np.float32).max),
        #                         'reward': array_spec.ArraySpec(shape=(), dtype=np.float32, name='reward'),
        #                         'step_type': array_spec.ArraySpec(shape=(), dtype=np.int32, name='step_type')})
        self._action_spec = array_spec.BoundedArraySpec(
            shape=(), dtype=np.int32, minimum=0, maximum=2, name='action')
        self._observation_spec = array_spec.BoundedArraySpec(
            shape=(12,), dtype=np.float32, minimum=np.finfo(np.float32).min,
            maximum=np.finfo(np.float32).max, name='observation')
        
        self.env = gym.make('SlimeVolley-v0')
        self._episode_ended = False
    def action_spec(self):
        return self._action_spec
    def observation_spec(self):
        return self._observation_spec
    
    def _reset(self):
        self._episode_ended = False
        return ts.restart(np.array(self.env.reset(), dtype=np.float32))
    def _step(self, action):
        if self._episode_ended:
            return self.reset()
        action_reshape = np.zeros(3)
        action_reshape[action] = 1
        observation, reward, episode_done, info = self.env.step(action_reshape)
        if episode_done:
            self._episode_ended = True
            return ts.termination(np.array(observation, dtype=np.float32), reward)
        else:
            return ts.transition(np.array(observation, dtype=np.float32), reward=reward, discount=1.0)
        
    
a = TFSlimeVolley()
tf_env = tf_py_environment.TFPyEnvironment(a)

In [6]:
#env_name = 'CartPole-v0'
#env = suite_gym.load(env_name)
#py_env = tf_py_environment.TFPyEnvironment(env)
#py_env.action_spec()
#time_spec = py_env.time_step_spec()
#py_env.reset()

#env = gym.make('SlimeVolley-v0')
#print("Observation Space " + str(env.observation_space.shape))
#print("Action Space " + str(env.action_space.shape))

In [7]:
#env.action_space.sample()
#env.observation_space.sample()
#env.reset()
#env.step([1,0,0])
#env.current_time_step()

In [8]:
train_env = tf_env
b = TFSlimeVolley()
eval_env = tf_py_environment.TFPyEnvironment(b)
#print(train_env.time_step_spec())
print(train_env.reset())

TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 12), dtype=float32, numpy=array([[ 1.2  ,  0.15 ,  0.   ,  0.   ,  0.   ,  1.2  , -0.686,  1.724,  1.2  ,  0.15 ,  0.   ,  0.   ]], dtype=float32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>})


In [9]:
utils.validate_py_environment(b, episodes=5)


# Deep Q Network Agent

In [10]:
# Create Q Deep NN.

fc_layer_params = (1600, 800, 400, 200, 100)
action_tensor_spec = tensor_spec.from_spec(train_env.action_spec())
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1

# Define a helper function to create Dense layers configured with the right
# activation and kernel initializer.
def dense_layer(num_units):
  return tf.keras.layers.Dense(
      num_units,
      activation=tf.keras.activations.relu,
      kernel_initializer=tf.keras.initializers.VarianceScaling(
          scale=2.0, mode='fan_in', distribution='truncated_normal'))

# QNetwork consists of a sequence of Dense layers followed by a dense layer
# with `num_actions` units to generate one q_value per available action as
# it's output.
dense_layers = [dense_layer(num_units) for num_units in fc_layer_params]
q_values_layer = tf.keras.layers.Dense(
    num_actions,
    activation=None,
    kernel_initializer=tf.keras.initializers.RandomUniform(
        minval=-0.03, maxval=0.03),
    bias_initializer=tf.keras.initializers.Constant(-0.2))
q_net = sequential.Sequential(dense_layers + [q_values_layer])

In [11]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter,
    target_update_period=20)

agent.initialize()

eval_policy = agent.policy
collect_policy = agent.collect_policy

# Random policy to collect initial data.
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),
                                                train_env.action_spec())

In [12]:
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=1,
    max_length=replay_buffer_max_length)

# Metrics and Evaluation

In [13]:
def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]

# See also the metrics module for standard implementations of different metrics.
# https://github.com/tensorflow/agents/tree/master/tf_agents/metrics

In [14]:
class BestModel:
    def __init__(self, agent):
        self.best_model = keras.models.clone_model(agent.model)
        self.best_model.set_weights(agent.model.get_weights())
        self.state_num = agent.state_num
        self.best_score = -10
        self.update_index = 0
    def UpdateBest(self, agent, eval_score, i):
        if (eval_score > self.best_score):
            self.best_model.set_weights(agent.model.get_weights())
            self.best_score = eval_score
            self.update_index = i
            print("New best!")
    def RunBest(self, state):
        state = np.reshape(state, [1, self.state_num])
        action = self.best_model(state, training=False)
        return np.argmax(action[0])

#bmodel = BestModel(agent)

## Run Training

In [15]:
def collect_step(environment, policy, buffer):
  time_step = environment.current_time_step()
  action_step = policy.action(time_step)
  next_time_step = environment.step(action_step.action)
  traj = trajectory.from_transition(time_step, action_step, next_time_step)

  # Add trajectory to the replay buffer
  buffer.add_batch(traj)

def collect_data(env, policy, buffer, steps):
  for _ in range(steps):
    collect_step(env, policy, buffer)

collect_data(train_env, random_policy, replay_buffer, initial_collect_steps)

# This loop is so common in RL, that we provide standard implementations. 
# For more details see tutorial 4 or the drivers module.
# https://github.com/tensorflow/agents/blob/master/docs/tutorials/4_drivers_tutorial.ipynb 
# https://www.tensorflow.org/agents/api_docs/python/tf_agents/drivers

# For the curious:
# Uncomment to peel one of these off and inspect it.
# iter(replay_buffer.as_dataset()).next()

# Dataset generates trajectories with shape [Bx2x...]
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3, 
    sample_batch_size=batch_size, 
    num_steps=2).prefetch(3)


dataset
iterator = iter(dataset)
print(iterator)
# For the curious:
# Uncomment to see what the dataset iterator is feeding to the agent.
# Compare this representation of replay data 
# to the collection of individual trajectories shown earlier.

# iterator.next()

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


In [16]:
try:
  %%time
except:
  pass

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)

# Reset the train step
agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
returns = [avg_return]

for _ in trange(num_iterations):

  # Collect a few steps using collect_policy and save to the replay buffer.
  collect_data(train_env, agent.collect_policy, replay_buffer, collect_steps_per_iteration)

  # Sample a batch of data from the buffer and update the agent's network.
  experience, unused_info = next(iterator)
  train_loss = agent.train(experience).loss

  step = agent.train_step_counter.numpy()

  if step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, train_loss))

  if step % eval_interval == 0:
    avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
    print('step = {0}: Average Return = {1}'.format(step, avg_return))
    returns.append(avg_return)

  0%|          | 0/90000 [00:00<?, ?it/s]

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))
step = 200: loss = 0.0009486199705861509
step = 400: loss = 0.00415068818256259
step = 600: loss = 0.0012521957978606224
step = 800: loss = 0.012919792905449867
step = 1000: loss = 0.013675099238753319
step = 1000: Average Return = -4.949999809265137
step = 1200: loss = 0.002205835422500968
step = 1400: loss = 0.02317243441939354
step = 1600: loss = 0.018308544531464577
step = 1800: loss = 0.019229987636208534
step = 2000: loss = 0.07849456369876862
step = 2000: Average Return = -4.699999809265137
step = 2200: loss = 0.007777783088386059
step = 2400: loss = 0.04006198048591614
step = 2600: loss = 0.00982610322535038
step = 2800: loss = 0.015280798077583313
step = 3000: loss = 0.019058309495449066
step = 3000: Average Return = -4.800000190734863
step = 3200

step = 32000: loss = 192.90353393554688
step = 32000: Average Return = -4.949999809265137
step = 32200: loss = 745.7275390625
step = 32400: loss = 503.27569580078125
step = 32600: loss = 483.1691589355469
step = 32800: loss = 652.2904052734375
step = 33000: loss = 649.485107421875
step = 33000: Average Return = -4.849999904632568
step = 33200: loss = 260.25640869140625
step = 33400: loss = 889.928955078125
step = 33600: loss = 181.34042358398438
step = 33800: loss = 397.6488342285156
step = 34000: loss = 266.4290771484375
step = 34000: Average Return = -4.800000190734863
step = 34200: loss = 1861.1512451171875
step = 34400: loss = 2875.189697265625
step = 34600: loss = 418.78741455078125
step = 34800: loss = 366.6003723144531
step = 35000: loss = 3285.595458984375
step = 35000: Average Return = -4.699999809265137
step = 35200: loss = 1659.505126953125
step = 35400: loss = 310.13037109375
step = 35600: loss = 3311.048095703125
step = 35800: loss = 279.54876708984375
step = 36000: loss =

step = 68000: Average Return = -4.849999904632568
step = 68200: loss = 370059.84375
step = 68400: loss = 327373.03125
step = 68600: loss = 3755084.75
step = 68800: loss = 94964.4453125
step = 69000: loss = 189866.09375
step = 69000: Average Return = -4.849999904632568
step = 69200: loss = 875707.375
step = 69400: loss = 181983.8125
step = 69600: loss = 11191054.0
step = 69800: loss = 7036163.0
step = 70000: loss = 1047173.9375
step = 70000: Average Return = -4.900000095367432
step = 70200: loss = 504424.9375
step = 70400: loss = 1022830.25
step = 70600: loss = 3924122.0
step = 70800: loss = 323931.46875
step = 71000: loss = 499378.5625
step = 71000: Average Return = -4.699999809265137
step = 71200: loss = 507634.0625
step = 71400: loss = 261663.34375
step = 71600: loss = 888309.0625
step = 71800: loss = 758320.375
step = 72000: loss = 4632989.5
step = 72000: Average Return = -4.949999809265137
step = 72200: loss = 383958.8125
step = 72400: loss = 437921.15625
step = 72600: loss = 37330

KeyboardInterrupt: 

In [ ]:
iterations = range(0, num_iterations + 1, eval_interval)
plt.plot(iterations, returns)
plt.ylabel('Average Return')
plt.xlabel('Iterations')
plt.ylim(top=5.5, bottom=-5.5)

## Watching a Game

In [ ]:
obs = env.reset()
done = False
total_reward = 0
img = plt.imshow(env.render(mode='rgb_array'))

while not done:
    action = bmodel.RunBest(obs)
    action_reshaped = np.zeros(agent.action_num)
    action_reshaped[action] = 1
    obs, reward, done, info = env.step(action_reshaped)
    total_reward += reward
    img.set_data(env.render(mode='rgb_array'))
    display.clear_output(wait=True)
    display.display(plt.gcf())

print("score:", total_reward)
print(bmodel.update_index)